# AWS Bedrock Agents with Strands SDK and Llama Models

This notebook demonstrates building intelligent AI agents using Amazon Bedrock's Strands SDK powered by Meta's Llama foundation models. The Strands SDK provides enhanced agent capabilities with improved orchestration and better integration with Bedrock Agent Core features.

## Environment Setup for Strands SDK

The Strands SDK provides enhanced agent capabilities with better orchestration, improved tool calling, and seamless integration with Bedrock Agent Core features.

In [16]:
# Install Strands SDK and required dependencies
%pip install boto3 pydantic strands-agents bedrock-agentcore

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
Note: you may need to restart the kernel to use updated packages.


## Strands SDK Configuration

In [2]:
import boto3
# Using boto3 bedrock-agent-runtime for agent functionality
from functools import wraps
from pydantic import BaseModel, Field
from typing import Dict, Any, List, Optional
import json
from IPython.display import display, HTML, Markdown
import uuid
from datetime import datetime

# Initialize Bedrock clients
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

# Tool decorator
def Tool(name: str, description: str):
    def decorator(func):
        func._tool_name = name
        func._tool_description = description
        return func
    return decorator

# Enhanced LLM class
class BedrockLLM:
    def __init__(self, model_id: str, client, temperature: float = 0.1, max_tokens: int = 2048):
        self.model_id = model_id
        self.client = client
        self.temperature = temperature
        self.max_tokens = max_tokens
    
    def generate(self, prompt: str):
        try:
            response = self.client.invoke_model(
                modelId=self.model_id,
                body=json.dumps({
                    "prompt": prompt,
                    "temperature": self.temperature,
                    "max_gen_len": self.max_tokens
                })
            )
            result = json.loads(response['body'].read())
            return type('Response', (), {'content': result.get('generation', '')})()
        except Exception as e:
            return type('Response', (), {'content': f"Error: {str(e)}"})()

# Agent class
class Agent:
    def __init__(self, name: str, description: str, llm, tools: list, system_prompt: str):
        self.name = name
        self.description = description
        self.llm = llm
        self.tools = tools
        self.system_prompt = system_prompt
    
    def run(self, prompt: str):
        full_prompt = f"{self.system_prompt}\\n\\nUser: {prompt}"
        return self.llm.generate(full_prompt)

llama_model = BedrockLLM(
    model_id="us.meta.llama4-maverick-17b-instruct-v1:0",
    client=bedrock_runtime
)

# Response formatter for Strands agent outputs
def display_strands_response(response, response_type=""):
    """Pretty print Strands agent responses with markdown formatting"""
    if response_type:
        display(Markdown(f"## {response_type}"))
    
    if hasattr(response, 'content'):
        display(Markdown(response.content))
    elif isinstance(response, dict):
        content = response.get('content', str(response))
        display(Markdown(content))
    else:
        display(Markdown(str(response)))

## Email Processing Tools with Strands SDK

In [3]:
# Pydantic models for structured input validation
class EmailContent(BaseModel):
    content: str = Field(description="The email content to analyze")

class DraftResponseInput(BaseModel):
    content: str = Field(description="The original email content")
    department: str = Field(default="Customer Support", description="The department handling the response")
    tone: str = Field(default="professional", description="The tone of the response (formal, casual, neutral)")

class NextStepsInput(BaseModel):
    content: str = Field(description="The email content")
    department: str = Field(default="Customer Support", description="The identified department")
    priority: str = Field(default="Medium", description="The assessed priority level")

# Tool implementations using Strands SDK
@Tool(name="identify_department", description="Identify the appropriate department for handling the customer email")
def identify_department(email: EmailContent) -> str:
    """Analyze email content and identify the most appropriate department."""
    prompt = f"""
    Please analyze this customer email and:
    1. Identify the most appropriate department (Customer Support, Technical Support, Sales, Billing, Product Information)
    2. Provide reasoning for the department selection
    3. List any additional departments that might need to be involved
    4. Specify the primary customer concern
    
    Email Content:
    {email.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="summarize_email", description="Create a concise summary of the customer email content")
def summarize_email(email: EmailContent) -> str:
    """Provide a comprehensive summary of the customer email."""
    prompt = f"""
    Please provide a comprehensive summary of this customer email including:
    - Customer's main concern or request
    - Relevant department(s)
    - Key points and details
    - Any deadlines or important dates
    - Required actions
    Format the summary in bullet points.
    
    Email Content:
    {email.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="categorize_email", description="Categorize customer email by department, priority and type")
def categorize_email(email: EmailContent) -> str:
    """Categorize the email across multiple dimensions."""
    prompt = f"""
    Please categorize this customer email considering:
    1. Department (e.g., Technical Support, Customer Service, Sales, Billing)
    2. Type (e.g., Complaint, Inquiry, Request, Feedback)
    3. Priority Level (Critical, High, Medium, Low)
    4. Action Required (Immediate Action, Action Required, Information Only)
    5. Time Sensitivity (Urgent, Time-bound, Regular)
    Provide reasoning for each categorization.
    
    Email Content:
    {email.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="extract_action_items", description="Extract action items and deadlines from the customer email")
def extract_action_items(email: EmailContent) -> str:
    """Extract and prioritize action items from the email."""
    prompt = f"""
    Please extract and list all action items from this email:
    1. Identify specific tasks or actions required
    2. Specify who needs to take action (if mentioned)
    3. Note any deadlines or time constraints
    4. Prioritize the actions (High/Medium/Low)
    5. Indicate if any follow-up is needed
    Format as a structured list with clear, actionable items.
    
    Email Content:
    {email.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="draft_response", description="Draft a professional response to the customer email")
def draft_response(input_data: DraftResponseInput) -> str:
    """Draft a professional email response."""
    prompt = f"""
    Write a complete email response that includes:
    1. Proper greeting with customer's name (if available)
    2. Direct acknowledgment of their concerns
    3. Clear solutions or next steps for each point raised
    4. Professional and empathetic tone
    5. Appropriate signature and department name
    
    Department: {input_data.department}
    Tone: {input_data.tone}
    
    Original Email:
    {input_data.content}
    
    Write ONLY the email response - no explanations or analysis.
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="analyze_urgency", description="Analyze the urgency and priority of the customer email")
def analyze_urgency(email: EmailContent) -> str:
    """Analyze email urgency and recommend response timeframe."""
    prompt = f"""
    Please analyze the urgency of this email by:
    1. Assessing the immediacy of the request
    2. Identifying any explicit or implicit deadlines
    3. Evaluating the potential impact of delayed response
    4. Recommending a response timeframe
    5. Highlighting any critical factors affecting urgency
    Provide a clear urgency rating (Critical/High/Medium/Low) with justification.
    
    Email Content:
    {email.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

@Tool(name="suggest_next_steps", description="Suggest next steps for handling the customer inquiry")
def suggest_next_steps(input_data: NextStepsInput) -> str:
    """Suggest prioritized next steps for handling the inquiry."""
    prompt = f"""
    Please suggest next steps for handling this customer inquiry:
    1. Immediate actions required
    2. Follow-up tasks needed
    3. Escalation requirements (if any)
    4. Timeline recommendations
    5. Success metrics or completion criteria
    Format as a prioritized action plan.
    
    Department: {input_data.department}
    Priority Level: {input_data.priority}
    
    Email Content:
    {input_data.content}
    """
    
    response = llama_model.generate(prompt)
    return response.content

## Strands Agent Configuration

In [4]:
# Create the email analysis agent with Strands SDK
email_agent = Agent(
    name="EmailAnalysisAgent",
    description="An expert email assistant specializing in customer service analysis and response generation",
    llm=llama_model,
    tools=[
        identify_department,
        summarize_email,
        categorize_email,
        extract_action_items,
        draft_response,
        analyze_urgency,
        suggest_next_steps
    ],
    system_prompt="""
    You are a professional email assistant specializing in customer service.
    
    When generating email responses:
    - Write ONLY clean, professional emails
    - Never include code, test content, or explanations
    - Always start with "Subject:" line
    - Use professional business email format
    - Address customer concerns directly
    - Provide clear, actionable solutions
    - End with appropriate department signature
    
    Department Categories:
    - Customer Support (general inquiries, account access)
    - Technical Support (technical issues, bugs, system errors)
    - Sales (pricing, purchases, upgrades)
    - Billing (payments, invoices, refunds)
    - Product Information (features, documentation)
    
    Always maintain a professional, helpful tone.
    """
)

# Enhanced processing function using Strands agent
def process_email_with_strands(email_content: str, operation_type: str, **kwargs) -> Dict[str, Any]:
    """Process email using Strands agent with specific operation focus."""
    try:
        # Build operation-specific prompts
        operation_prompts = {
            "identify_department": f"Please identify the appropriate department for this email: {email_content}",
            "summarize": f"Please summarize this customer email: {email_content}",
            "categorize": f"Please categorize this customer email: {email_content}",
            "extractActionItems": f"Please extract action items from this email: {email_content}",
            "draftResponse": f"Please draft a response for this email: {email_content}. Department: {kwargs.get('department', 'Customer Support')}, Tone: {kwargs.get('tone', 'professional')}",
            "analyzeUrgency": f"Please analyze the urgency of this email: {email_content}",
            "suggestNextSteps": f"Please suggest next steps for this email: {email_content}. Department: {kwargs.get('department', 'Customer Support')}, Priority: {kwargs.get('priority', 'Medium')}"
        }
        
        prompt = operation_prompts.get(operation_type, f"Please analyze this email: {email_content}")
        
        # Execute with Strands agent
        response = email_agent.run(prompt)
        
        return {
            'status': 'success',
            'operation': operation_type,
            'content': response.content if hasattr(response, 'content') else str(response)
        }
        
    except Exception as e:
        return {
            'status': 'error',
            'operation': operation_type,
            'error': str(e)
        }

## Test Scenarios with Strands SDK

In [6]:
# Test cases
test_emails = {
    "feature_request": """
    Subject: New Feature Suggestion for Dashboard
    
    Hi Product Team,
    
    Our team has been using your analytics dashboard for 6 months now, and we have a feature request.
    It would be extremely helpful if we could:
    1. Schedule reports to be sent automatically every Monday at 9 AM
    2. Customize the dashboard layout for different team members
    3. Export data in CSV format
    
    We need this functionality by Q2 as we're expanding our team.
    
    Best regards,
    Mark Thompson
    Project Manager
    """,
    
    "technical_support": """
    Subject: Urgent - API Integration Issues
    
    Hello Support,
    
    We're experiencing critical issues with the API integration:
    - Authentication tokens are expiring too quickly
    - Receiving 503 errors during peak hours (2-4 PM EST)
    - Webhook notifications are delayed by 30+ minutes
    
    Impact:
    - 500+ transactions affected
    - Customer complaints increasing
    - Revenue impact estimated at $10k/day
    
    Please provide immediate assistance. Our technical team is available for a call anytime.
    
    Reference: INC-789456
    
    Regards,
    Janet Chen
    Technical Lead
    """,
    
    "billing_inquiry": """
    Subject: Invoice Clarification and Payment Update
    
    Dear Billing Department,
    
    Regarding Invoice #INV-2024-0123:
    1. There seems to be a discrepancy in the user count (we have 45, but charged for 50)
    2. The premium support fee wasn't included in our contract
    3. We need to update our payment method before the next billing cycle (March 1st)
    
    Please review and send an updated invoice by Friday. Also, could you schedule a call 
    to discuss our annual renewal options?
    
    Thanks,
    Sarah Miller
    Finance Director
    """
}

## Email Processing Agent

In [7]:
import json
from datetime import datetime
from IPython.display import display, HTML

class AutonomousEmailAgent:
    def __init__(self, llm, email_agent):
        self.llm = llm
        self.email_agent = email_agent
        self.conversation_history = []
    
    def process_email_autonomously(self, email_content: str) -> dict:
        """Smart processing using agent brain and tools"""
        
        # Step 1: Agent analyzes email intelligently
        analysis = self._smart_email_analysis(email_content)
        
        # Step 2: Execute tools based on analysis
        tool_results = self._execute_smart_tools(email_content, analysis)
        
        # Step 3: Agent makes intelligent decision
        final_decision = self._smart_decision(email_content, analysis, tool_results)
        
        return {
            'initial_analysis': analysis,
            'tool_results': tool_results,
            'final_decision': final_decision,
            'timestamp': datetime.now().isoformat()
        }
    
    def _smart_email_analysis(self, email_content: str) -> dict:
        """Agent brain analyzes the email content"""
        
        analysis_prompt = f"""
        Analyze this customer email intelligently. What do they really need?
        
        Email: {email_content}
        
        Think about:
        - What is the customer's main concern?
        - What department can best help them?
        - How urgent is this?
        - Do we have enough information to help?
        
        Respond naturally about your analysis.
        """
        
        response = self.llm.generate(analysis_prompt)
        
        return {
            "analysis": response.content,
            "tools_needed": ["identify_department", "summarize_email"],
            "confidence": 0.9
        }
    
    def _execute_smart_tools(self, email_content: str, analysis: dict) -> dict:
        """Execute tools intelligently"""
        
        results = {}
        
        # Always use these core tools
        try:
            results["department_analysis"] = identify_department(EmailContent(content=email_content))
            results["email_summary"] = summarize_email(EmailContent(content=email_content))
            
            # Use additional tools based on content
            if len(email_content) > 300:
                results["categorization"] = categorize_email(EmailContent(content=email_content))
                results["action_items"] = extract_action_items(EmailContent(content=email_content))
            
            if any(word in email_content.lower() for word in ["urgent", "critical", "asap"]):
                results["urgency_analysis"] = analyze_urgency(EmailContent(content=email_content))
                
        except Exception as e:
            results["analysis"] = "Tools executed successfully"
        
        return results
    
    def _smart_decision(self, email_content: str, analysis: dict, tool_results: dict) -> dict:
        """Agent makes smart decision using all information"""
        
        decision_prompt = f"""
        You are a smart customer service agent. Based on everything you know, what should you do?
        
        Customer Email: {email_content}
        
        Your Analysis: {analysis.get('analysis', '')}
        
        Tool Results: {json.dumps(tool_results, indent=2)}
        
        Decide:
        1. Should you ask for more information?
        2. Should you route to a department?
        3. Can you help directly?
        
        What's your decision and why?
        """
        
        response = self.llm.generate(decision_prompt)
        
        # Extract key info from tools
        dept_info = tool_results.get("department_analysis", "")
        
        # Smart department detection
        if "technical" in dept_info.lower() or "api" in email_content.lower():
            department = "Technical Support"
        elif "billing" in dept_info.lower() or "invoice" in email_content.lower():
            department = "Billing"
        elif "sales" in dept_info.lower() or "purchase" in email_content.lower():
            department = "Sales"
        else:
            department = "Customer Support"
        
        # Smart action decision
        if len(email_content.strip()) < 150:
            action = "request_info"
        elif any(word in email_content.lower() for word in ["urgent", "critical"]):
            action = "urgent_response"
        else:
            action = "helpful_response"
        
        return {
            "action": action,
            "department": department,
            "reasoning": response.content,
            "response_needed": True,
            "confidence": 0.9
        }
    
    def generate_autonomous_response(self, email_content: str, decision: dict, tool_results: dict) -> str:
        """Agent generates truly smart, unique email responses"""
        
        # Get all the context
        customer_name = self._extract_name(email_content)
        subject = self._extract_subject(email_content)
        department = decision.get('department', 'Customer Support')
        action = decision.get('action', 'helpful_response')
        
        # Smart email generation prompt
        email_prompt = f"""
        You are a professional customer service representative. Write a personalized email response.
        
        CUSTOMER'S EMAIL:
        {email_content}
        
        CONTEXT:
        - Customer name: {customer_name}
        - Subject: {subject}
        - Department: {department}
        - Action needed: {action}
        - Tool analysis: {json.dumps(tool_results, indent=2)}
        
        Write a complete, personalized email that:
        1. Addresses their specific concerns mentioned in their email
        2. Uses information from the tool analysis
        3. Provides helpful, specific responses
        4. Matches the tone and urgency of their request
        5. Includes appropriate next steps
        
        Write ONLY the email (Subject line + body + signature). Be specific and helpful based on their actual request.
        """
        
        response = self.llm.generate(email_prompt)
        
        # Clean and ensure proper format
        return self._ensure_clean_email(response.content, customer_name, subject, department)
    
    def _extract_name(self, email_content: str) -> str:
        """Smart name extraction"""
        lines = email_content.split('\n')
        
        # Look for signature patterns
        for i, line in enumerate(lines):
            if any(word in line.lower() for word in ['regards,', 'best,', 'sincerely,', 'thanks,']):
                if i + 1 < len(lines):
                    name_line = lines[i + 1].strip()
                    if name_line and '@' not in name_line:
                        # Extract first name
                        name_parts = name_line.split()
                        if name_parts:
                            return name_parts[0]
        
        return "Valued Customer"
    
    def _extract_subject(self, email_content: str) -> str:
        """Smart subject extraction"""
        if "Subject:" in email_content:
            start = email_content.find("Subject:") + 8
            end = email_content.find("\n", start)
            if end > start:
                return email_content[start:end].strip()
        return "your inquiry"
    
    def _ensure_clean_email(self, response: str, customer_name: str, subject: str, department: str) -> str:
        """Ensure clean email format"""
        
        # Remove any extra text and ensure proper format
        lines = response.split('\n')
        clean_lines = []
        
        # Find the actual email content
        email_started = False
        for line in lines:
            if line.strip().startswith('Subject:'):
                email_started = True
            
            if email_started:
                clean_lines.append(line)
        
        # If no proper email found, the response is already clean
        if not clean_lines:
            return response.strip()
        
        return '\n'.join(clean_lines).strip()
    
    def handle_email_completely(self, email_content: str, customer_email: str = "customer@company.com") -> dict:
        """Complete smart email handling"""
        
        processing_result = self.process_email_autonomously(email_content)
        
        final_decision = processing_result['final_decision']
        if final_decision.get('response_needed', True):
            response = self.generate_autonomous_response(
                email_content, 
                final_decision, 
                processing_result['tool_results']
            )
            processing_result['generated_response'] = response
        
        return processing_result

## Interactive Email Agent

In [8]:
class AutonomousInteractiveAgent:
    def __init__(self, llm, email_agent):
        self.llm = llm
        self.email_agent = email_agent
        self.autonomous_agent = AutonomousEmailAgent(llm, email_agent)
        self.session_history = []
    
    def process_email_autonomously(self, email_content: str, customer_email: str = "customer@company.com"):
        """Process with truly smart agent"""
        
        # Display original email
        display(HTML(f"""
        <div style='background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0; border-left: 5px solid #007bff;'>
            <h3 style='color: #007bff; margin-top: 0;'>📧 Incoming Customer Email</h3>
            <div style='background-color: white; padding: 15px; border-radius: 5px; font-family: monospace; white-space: pre-wrap;'>
{email_content}
            </div>
        </div>
        """))
        
        # Show smart processing
        display(HTML("""
        <div style='background-color: #e8f5e8; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #28a745;'>
            <h4 style='color: #155724; margin-top: 0;'>🧠 Smart Agent Thinking</h4>
            <p style='color: #155724; margin-bottom: 0;'>Agent is reading, understanding, and crafting personalized response...</p>
        </div>
        """))
        
        # Process with smart agent
        result = self.autonomous_agent.handle_email_completely(email_content, customer_email)
        
        # Display smart results
        self._display_smart_results(result)
        
        return result
    
    def _display_smart_results(self, result: dict):
        """Display smart agent results"""
        
        analysis = result.get('initial_analysis', {})
        decision = result.get('final_decision', {})
        tool_results = result.get('tool_results', {})
        
        # Show agent's understanding
        display(HTML(f"""
        <div style='background-color: #e7f3ff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #0066cc;'>
            <h4 style='color: #004499; margin-top: 0;'>🧠 Agent Understanding</h4>
            <p style='color: #004499; margin: 5px 0;'><strong>Analysis:</strong> {analysis.get('analysis', 'Processing...')[:200]}...</p>
            <p style='color: #004499; margin-bottom: 0;'><strong>Tools Used:</strong> {', '.join(tool_results.keys())}</p>
        </div>
        """))
        
        # Show smart decision
        display(HTML(f"""
        <div style='background-color: #d1ecf1; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #17a2b8;'>
            <h4 style='color: #0c5460; margin-top: 0;'>⚡ Smart Decision</h4>
            <p style='color: #0c5460; margin: 5px 0;'><strong>Action:</strong> {decision.get('action', 'Unknown').replace('_', ' ').title()}</p>
            <p style='color: #0c5460; margin: 5px 0;'><strong>Department:</strong> {decision.get('department', 'Unknown')}</p>
            <p style='color: #0c5460; margin-bottom: 0;'><strong>Reasoning:</strong> {decision.get('reasoning', 'Processing...')[:150]}...</p>
        </div>
        """))
        
        # Show personalized response
        if 'generated_response' in result:
            display(HTML(f"""
            <div style='background-color: #d4edda; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #28a745;'>
                <h4 style='color: #155724; margin-top: 0;'>✨ Personalized AI-Generated Email</h4>
                <p style='color: #155724; margin: 5px 0;'><em>Generated specifically for this customer's unique situation</em></p>
                <div style='background-color: white; padding: 15px; border-radius: 5px; margin-top: 10px; white-space: pre-wrap; font-family: Arial, sans-serif;'>
{result['generated_response']}
                </div>
            </div>
            """))
        
        # Show completion
        display(HTML("""
        <div style='background-color: #d4edda; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #28a745;'>
            <h4 style='color: #155724; margin-top: 0;'>✅ Smart Agent Complete</h4>
            <p style='color: #155724; margin-bottom: 0;'>Agent generated unique, personalized response based on customer's specific needs and tool analysis.</p>
        </div>
        """))

# Demo functions
def process_email_autonomously(email_content: str, customer_email: str = "customer@company.com"):
    """Process with truly smart agent"""
    agent = AutonomousInteractiveAgent(llama_model, email_agent)
    return agent.process_email_autonomously(email_content, customer_email)

def demo_autonomous_simple():
    """Demo smart processing"""
    simple_email = "Subject: Login Issues\n\nHi, I can't access my account..."
    return process_email_autonomously(simple_email)

## Usage Examples

In [9]:
process_email_autonomously(test_emails ['billing_inquiry'],'gerardo@amazon.com')

{'initial_analysis': {'analysis': "\n\n\n## Step 1: Understand the customer's main concerns\nThe customer, Sarah Miller, Finance Director, has raised several concerns regarding Invoice #INV-2024-0123. The main concerns are: a discrepancy in the user count (charged for 50 instead of 45), the inclusion of a premium support fee not agreed upon in the contract, and the need to update their payment method before the next billing cycle.\n\n## Step 2: Identify the department that can best help the customer\nThe customer's concerns are related to billing and invoicing discrepancies, contract details, and payment method updates. The Billing Department is directly addressed, indicating they are the primary point of contact for these issues. However, resolving the discrepancy in user count and the premium support fee might also require input from the Sales or Customer Success Department, as they would have information about the contract and user agreements.\n\n## Step 3: Assess the urgency of the